In [1]:

import numpy as np
import pandas as pd
from scripts.unsupervised_utils.filter_by_noise_ratio import filter_by_noise_ratio
from scripts.unsupervised_utils.paretos import extract_pareto_front
from scripts.Training.unsupervised.grid_search_hdbscan import run_hdbscan_grid
from scripts.Training.unsupervised.grid_search_knn import run_kmeans_grid
from scripts.Training.unsupervised.run_hdbscan_combinaison import  run_hdbscan_combinations
from scripts.Training.unsupervised.grid_search_clarans import run_clarans_grid
from scripts.Training.unsupervised.run_algo import kmeans_with_map , hdbscan_with_map

## Load data

In [3]:

# -----------------------------
# 1. Load your data
# -----------------------------
data_path_32 = "../data/preprocessed/preprocessed_reduced_unsupervised_32.csv"
X_data_32 = pd.read_csv(data_path_32)



data_path_10 = "../data/preprocessed/preprocessed_reduced_unsupervised_10.csv"
X_data_10 = pd.read_csv(data_path_10)

shapefile_path = "../data/shapefiles/combined/north/alg_tun_north.shp"


print("Data shape after sampling (k-mean):", X_data_32.shape)
print("Data shape after sampling (dbscan):", X_data_10.shape)

Data shape after sampling (k-mean): (735483, 32)
Data shape after sampling (dbscan): (735483, 10)


## K-mean

In [21]:
results = run_kmeans_grid(
    X=X_data_32,
    k_values=list(range(3, 51)),  # 3,5,7,...,49
    output_path="./results/knn_metrics.csv",
    algorithm="minibatch_kmeans",  
    batch_size=4096,
    metrics=("ch", "dbi","wcss"),
)



minibatch_kmeans Grid Search: 100%|██████████| 48/48 [02:12<00:00,  2.76s/it]


In [ ]:
pareto = extract_pareto_front(
    csv_path="./results/knn_metrics.csv",
    metrics=["CH", "DBI"],
)

print(pareto)

    K       init         algorithm             CH       DBI  WCSS_per_point
0   3  k-means++  minibatch_kmeans  254988.834430  1.721380       16.684688
1   5  k-means++  minibatch_kmeans  180414.047959  1.701579       14.263527
2   8  k-means++  minibatch_kmeans  147410.138872  1.512309       11.756861
3  11  k-means++  minibatch_kmeans  126335.821244  1.505054       10.413611
4  12  k-means++  minibatch_kmeans  125704.384579  1.368123        9.811640
5  13  k-means++  minibatch_kmeans  124980.671951  1.341546        9.296829
6  14  k-means++  minibatch_kmeans  120858.929637  1.329219        9.021307
7  16  k-means++  minibatch_kmeans  115956.241494  1.277816        8.403590
8  21  k-means++  minibatch_kmeans  117611.923318  1.288624        6.730543
9  26  k-means++  minibatch_kmeans  110334.797442  1.211595        5.948190


In [32]:
results = run_kmeans_grid(
    X=X_data_32,
    k_values=[3,5,8,11,12,14,16,21,26],  # 3,5,7,...,49
    n_init=10,
    output_path="./results/knn_silhouette.csv",
    algorithm="minibatch_kmeans",  
    batch_size=4096,
    metrics=("silhouette"),
)


minibatch_kmeans Grid Search: 100%|██████████| 9/9 [55:06<00:00, 367.38s/it]


In [4]:
gdf_km, km_metrics = kmeans_with_map(
    df=X_data_32,
    shapefile_path=shapefile_path,
    k_value=21,
    algorithm="minibatch_kmeans",

)




KeyError: "['latitude'] not found in axis"

## HDBSCAN 

In [ ]:
results = run_hdbscan_grid(
    X=X_data_10,
    min_cluster_sizes=[50, 100, 200, 500, 1000],
    min_samples_values=[5, 10, 20, 50],
    output_path="./results/hdbscan_results.csv",
    metrics=("ch", "dbi", "wcss"),
)


HDBSCAN Grid Search:   5%|▌         | 1/20 [09:51<3:07:26, 591.90s/it]

{'algorithm': 'hdbscan', 'min_cluster_size': 50, 'min_samples': 5, 'n_clusters': 4896, 'n_noise': np.int64(132861), 'noise_ratio': np.float64(0.18064455602644794), 'CH': 9171.463955278918, 'DBI': 0.8613381514160154, 'WCSS': ae_feature_0    1904.976222
ae_feature_1    2187.019630
ae_feature_2    2610.673438
ae_feature_3    1770.657166
ae_feature_4    1794.338724
ae_feature_5    2028.925024
ae_feature_6    1534.773417
ae_feature_7    1365.848959
ae_feature_8    3307.210763
ae_feature_9    1770.328046
dtype: float64}


HDBSCAN Grid Search:  10%|█         | 2/20 [19:08<2:51:24, 571.36s/it]

{'algorithm': 'hdbscan', 'min_cluster_size': 50, 'min_samples': 10, 'n_clusters': 4705, 'n_noise': np.int64(141521), 'noise_ratio': np.float64(0.19241913137353275), 'CH': 9613.115812421172, 'DBI': 0.8402605206507678, 'WCSS': ae_feature_0    1637.425615
ae_feature_1    2211.965408
ae_feature_2    2542.972394
ae_feature_3    1769.443822
ae_feature_4    1698.391460
ae_feature_5    2122.254359
ae_feature_6    1459.864554
ae_feature_7    1353.819022
ae_feature_8    3033.734207
ae_feature_9    1665.027780
dtype: float64}


HDBSCAN Grid Search:  15%|█▌        | 3/20 [28:13<2:38:23, 559.06s/it]

...

{'algorithm': 'hdbscan', 'min_cluster_size': 1000, 'min_samples': 10, 'n_clusters': 186, 'n_noise': np.int64(87669), 'noise_ratio': np.float64(0.1191992201043396), 'CH': 27565.36467344073, 'DBI': 1.6213186785981404, 'WCSS': ae_feature_0    16371.552735
ae_feature_1    27906.379734
ae_feature_2    26935.896712
ae_feature_3    16226.320915
ae_feature_4    14295.305269
ae_feature_5    18406.904630
ae_feature_6    10432.204091
ae_feature_7    10395.578268
ae_feature_8    22010.647658
ae_feature_9    15433.984887
dtype: float64}


HDBSCAN Grid Search:  95%|█████████▌| 19/20 [1:46:25<04:29, 269.51s/it]

{'algorithm': 'hdbscan', 'min_cluster_size': 1000, 'min_samples': 20, 'n_clusters': 185, 'n_noise': np.int64(95088), 'noise_ratio': np.float64(0.12928646889187106), 'CH': 27930.08263966517, 'DBI': 1.5998466732280687, 'WCSS': ae_feature_0    15918.056955
ae_feature_1    27357.436802
ae_feature_2    26526.519074
ae_feature_3    15814.317662
ae_feature_4    13712.543237
ae_feature_5    18114.901448
ae_feature_6     9444.075308
ae_feature_7    10147.271740
ae_feature_8    21119.397391
ae_feature_9    14702.237520
dtype: float64}


HDBSCAN Grid Search: 100%|██████████| 20/20 [1:50:58<00:00, 332.92s/it]

{'algorithm': 'hdbscan', 'min_cluster_size': 1000, 'min_samples': 50, 'n_clusters': 174, 'n_noise': np.int64(107191), 'noise_ratio': np.float64(0.1457423217123985), 'CH': 29329.604730543622, 'DBI': 1.4589200360601353, 'WCSS': ae_feature_0    14375.044213
ae_feature_1    27391.152312
ae_feature_2    25517.505758
ae_feature_3    15262.104495
ae_feature_4    13019.377237
ae_feature_5    17882.186885
ae_feature_6     7959.508951
ae_feature_7    10431.774058
ae_feature_8    21287.833577
ae_feature_9    13197.291996
dtype: float64}


In [5]:
filter_by_noise_ratio("./results/hdbscan_results.csv","./results/hdbscan_results_filtered.csv",min_noise_ratio=0.2)

Saved 18 / 20 rows to ./results/hdbscan_results_filtered.csv


In [8]:
pareto = extract_pareto_front(
    csv_path="./results/hdbscan_results_filtered.csv",
    metrics=["CH", "DBI"],
)

print(pareto)

  algorithm  min_cluster_sizes  min_samples  n_clusters  n_noise  noise_ratio  \
0   hdbscan                 50           10        4705   141521     0.192419   
1   hdbscan                100           50        1117   133634     0.181696   
2   hdbscan                200           20         661    88982     0.120984   
3   hdbscan                200           50         550    97745     0.132899   
4   hdbscan                500           50         295    93779     0.127507   
5   hdbscan               1000           50         174   107191     0.145742   

             CH       DBI  
0   9613.115812  0.840261  
1  12354.581973  0.955520  
2  15974.816742  1.205005  
3  15464.683693  1.118552  
4  24703.715102  1.334042  
5  29329.604731  1.458920  


In [ ]:
param_combinations = [
    {"min_cluster_size": 50, "min_samples": 10},
    {"min_cluster_size": 100, "min_samples": 50},
    {"min_cluster_size": 200, "min_samples": 20},
    {"min_cluster_size": 200, "min_samples": 50},
    {"min_cluster_size": 500, "min_samples": 50},
    {"min_cluster_size": 1000, "min_samples": 50},
    
]

df = run_hdbscan_combinations(
    X=X_data_10,
    param_combinations=param_combinations,
    output_path="./results/hdbscan_selected_runs.csv",
    metrics=("ch", "dbi","silhouette"),
)


HDBSCAN Runs:  17%|█▋        | 1/6 [12:31<1:02:36, 751.36s/it]

{'algorithm': 'hdbscan', 'min_cluster_size': 50, 'min_samples': 10, 'n_clusters': 4705, 'n_noise': np.int64(141521), 'noise_ratio': np.float64(0.19241913137353275), 'CH': 9613.115812421172, 'DBI': 0.8402605206507678, 'Silhouette': np.float64(0.415047631999162), 'Silhouette_std': np.float64(0.0012099766964917239)}


HDBSCAN Runs:  33%|███▎      | 2/6 [21:20<41:23, 620.80s/it]  

{'algorithm': 'hdbscan', 'min_cluster_size': 100, 'min_samples': 50, 'n_clusters': 1117, 'n_noise': np.int64(133634), 'noise_ratio': np.float64(0.1816955660429949), 'CH': 12354.581972751314, 'DBI': 0.9555195940977038, 'Silhouette': np.float64(0.24981478331024148), 'Silhouette_std': np.float64(0.001807645142020548)}


HDBSCAN Runs:  50%|█████     | 3/6 [29:19<27:47, 555.78s/it]

...

{'algorithm': 'hdbscan', 'min_cluster_size': 200, 'min_samples': 50, 'n_clusters': 550, 'n_noise': np.int64(97745), 'noise_ratio': np.float64(0.13289906088923878), 'CH': 15464.683692850305, 'DBI': 1.1185523937460142, 'Silhouette': np.float64(0.22099316894987764), 'Silhouette_std': np.float64(0.0016330261896091833)}


HDBSCAN Runs:  83%|████████▎ | 5/6 [44:55<08:20, 500.89s/it]

{'algorithm': 'hdbscan', 'min_cluster_size': 500, 'min_samples': 50, 'n_clusters': 295, 'n_noise': np.int64(93779), 'noise_ratio': np.float64(0.12750668608247914), 'CH': 24703.715102483402, 'DBI': 1.3340416980474894, 'Silhouette': np.float64(0.22021246746883166), 'Silhouette_std': np.float64(0.0010693207788540038)}


HDBSCAN Runs: 100%|██████████| 6/6 [52:35<00:00, 526.00s/it]

{'algorithm': 'hdbscan', 'min_cluster_size': 1000, 'min_samples': 50, 'n_clusters': 174, 'n_noise': np.int64(107191), 'noise_ratio': np.float64(0.1457423217123985), 'CH': 29329.604730543622, 'DBI': 1.4589200360601353, 'Silhouette': np.float64(0.20762483780073993), 'Silhouette_std': np.float64(0.001571614176482901)}


In [ ]:
gdf_hdb, hdb_metrics = hdbscan_with_map(
    df=X_data_10,
    shapefile_path="algeria_regions.shp",
    min_cluster_size=300,
    min_samples=50,
)

## Clarans

In [ ]:
results = run_clarans_grid(
    X=X_data_32.values,
    k_values=range(3, 21, 2),
    numlocal_values=[3, 5],
    maxneighbor_values=[20, 50],
    output_path="./results/clarans_results.csv",
    metrics=("ch", "dbi", "avg_medoid_dist"),
)


CLARANS Grid Search:   0%|          | 0/36 [00:00<?, ?it/s]